In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
import Captcha_seg
import time

importing Jupyter notebook from Captcha_seg.ipynb


In [2]:
CATEGORIES =[]
for i in os.listdir("captcha_segmented/"):
    CATEGORIES.append(i)
print(CATEGORIES)

['6', 'x', '3', 'd', '4', 'y', 'm', '8', 'c', 'n', '5', 'f', 'b', 'w', '7', 'p', '2', 'e', 'g']


In [3]:
SEGMENTS_DIR = 'Segmented'
CAPTCHA_DIR = 'test/'
predictions = []
b_captcha = []

DATADIR = "./captcha_segmented/"
#CATEGORIES= ["2","3","4","5","6","7","8","b","c","d","e","f","g","m","n","p","w","x","y"]
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        #plt.imshow(img_array, cmap="gray")
        break
    break

In [4]:
print(img_array.shape)

(38, 24)


In [5]:
IMG_SIZE = 50

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#plt.imshow(new_array, cmap= 'gray')
#plt.show()

In [6]:
train_data = []

def create_train_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                train_data.append([new_array,class_num])
            except Exception as e:
                pass
            
create_train_data()

In [7]:
print(len(train_data))

3010


In [8]:
import random

random.shuffle(train_data)

In [9]:
for sample in train_data[:10]:
    print(sample[1])

4
9
3
4
3
14
3
8
12
2


In [10]:
X = []
y = []

In [11]:
for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [12]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [13]:
pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

(50, 50, 1)

In [14]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 2408 samples, validate on 602 samples
Epoch 1/20
2408/2408 [==============================] - 17s 7ms/step - loss: 2.9227 - acc: 0.0752 - val_loss: 2.8810 - val_acc: 0.0947
Epoch 2/20
2408/2408 [==============================] - 16s 7ms/step - loss: 2.7693 - acc: 0.1507 - val_loss: 2.5810 - val_acc: 0.1960
Epoch 3/20
2408/2408 [==============================] - 16s 7ms/step - loss: 2.3812 - acc: 0.2749 - val_loss: 2.1734 - val_acc: 0.3306
Epoch 4/20
2408/2408 [==============================] - 17s 7ms/step - loss: 1.9815 - acc: 0.4020 - val_loss: 1.8755 - val_acc: 0.4319
Epoch 5/20
2408/2408 [==============================] - 16s 7ms/step - loss: 1.6539 - acc: 0.4963 - val_loss: 1.6221 - val_acc: 0.5316
Epoch 6/20
2408/2408 [==============================] - 16s 7ms/step - loss: 1.4363 - acc: 0.5673 - val_loss: 1.5011 - val_acc: 0.5648
Epoch 7/20
2408/2408 [==============================] - 16s 7ms/step - loss: 1.2244 - acc: 0.6296 - val_loss: 1.3949 - val_acc: 0.5980
Epoch 8/

In [15]:
loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

3010/3010 [==============================] - 6s 2ms/step


In [16]:
print(loss_value, metrics_value)

0.3345386809104763 0.9362126226441012


In [17]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
for captchas in os.listdir(CAPTCHA_DIR):
    start_time = time.time()
    predict_list= []
    captcha_name = captchas.split('.')
    print('\n')
    print('Actual: ',captcha_name[0])
    
    captcha_path =os.path.join(CAPTCHA_DIR,captchas)
    Captcha_seg.segment(captcha_path)
    
    files = []
    for segments in os.listdir(SEGMENTS_DIR):
        files.append(segments)
        files.sort()
    
    for segments in files:
        filepath = os.path.join(SEGMENTS_DIR,segments)
        prediction = model.predict([prepare(filepath)])
        predict_list.append(CATEGORIES[np.argmax(prediction[0])])
    
    elapsed_time = time.time()-start_time
    total_time += elapsed_time
    final_prediction = ''.join(predict_list)
    if captcha_name[0]== final_prediction:
        score +=1
    
    print('Predicted:',final_prediction,'\n')
    print('Percentage Similarity= ',similarity(captcha_name[0],final_prediction))
    print('Elapsed time= ',elapsed_time,'s')
    print('<------------------------------->')    
    predictions.append(similarity(captcha_name[0],final_prediction))
    count += 1


print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',count)
avg_prediction = score/count
print('Average Characters Predicted= ',avg_prediction,'%')
print('Total time taken= ',total_time,'s')
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  d22n7
Predicted: d25n7 

Percentage Similarity=  80.0
Elapsed time=  0.24348998069763184 s
<------------------------------->


Actual:  3den6
Predicted: 3b5y5 

Percentage Similarity=  20.0
Elapsed time=  0.01547861099243164 s
<------------------------------->


Actual:  cc845
Predicted: 5c345 

Percentage Similarity=  60.0
Elapsed time=  0.01450657844543457 s
<------------------------------->


Actual:  bc8nf
Predicted: bc3nf 

Percentage Similarity=  80.0
Elapsed time=  0.014637947082519531 s
<------------------------------->


Actual:  eppg3
Predicted: eppg3 

Percentage Similarity=  100.0
Elapsed time=  0.014490604400634766 s
<------------------------------->


Actual:  cy3nw
Predicted: e73yx 

Percentage Similarity=  20.0
Elapsed time=  0.015045642852783203 s
<------------------------------->


Actual:  dcnp8
Predicted: e2ne8 

Percentage Similarity=  40.0
Elapsed time=  0.015209674835205078 s
<------------------------------->


Actual:  4d22m
Predicted: 4d22c 

Percent

Predicted: 5gnm3 

Percentage Similarity=  80.0
Elapsed time=  0.018796205520629883 s
<------------------------------->


Actual:  268g2
Predicted: 753g2 

Percentage Similarity=  40.0
Elapsed time=  0.018405437469482422 s
<------------------------------->


Actual:  6e2dg
Predicted: 6e22e 

Percentage Similarity=  60.0
Elapsed time=  0.016509532928466797 s
<------------------------------->


Actual:  7fde7
Predicted: 7f32y 

Percentage Similarity=  40.0
Elapsed time=  0.015223503112792969 s
<------------------------------->


Actual:  5n3w4
Predicted: 5mxw4 

Percentage Similarity=  60.0
Elapsed time=  0.014777660369873047 s
<------------------------------->


Actual:  62fgn
Predicted: 52fgn 

Percentage Similarity=  80.0
Elapsed time=  0.015439033508300781 s
<------------------------------->


Actual:  65m85
Predicted: 65y85 

Percentage Similarity=  80.0
Elapsed time=  0.015014410018920898 s
<------------------------------->


Actual:  573d8
Predicted: 573b5 

Percentage Similarity=

Predicted: 7b4xn 

Percentage Similarity=  60.0
Elapsed time=  0.018091678619384766 s
<------------------------------->


Actual:  5x7x5
Predicted: 5x7x5 

Percentage Similarity=  100.0
Elapsed time=  0.017884016036987305 s
<------------------------------->


Actual:  6pfy4
Predicted: 5p7y4 

Percentage Similarity=  60.0
Elapsed time=  0.015553474426269531 s
<------------------------------->


Actual:  76nxn
Predicted: c7nxn 

Percentage Similarity=  60.0
Elapsed time=  0.015522480010986328 s
<------------------------------->


Actual:  b3xpn
Predicted: b33pn 

Percentage Similarity=  80.0
Elapsed time=  0.015624046325683594 s
<------------------------------->


Actual:  befbd
Predicted: pefyd 

Percentage Similarity=  60.0
Elapsed time=  0.014890909194946289 s
<------------------------------->


Actual:  4n3mn
Predicted: en3yx 

Percentage Similarity=  40.0
Elapsed time=  0.015251398086547852 s
<------------------------------->


Actual:  de7f8
Predicted: de725 

Percentage Similarity

Predicted: e4gg2 

Percentage Similarity=  60.0
Elapsed time=  0.014822959899902344 s
<------------------------------->


Actual:  6pwcn
Predicted: 5pwcy 

Percentage Similarity=  60.0
Elapsed time=  0.018968820571899414 s
<------------------------------->


Actual:  43gey
Predicted: 435yy 

Percentage Similarity=  60.0
Elapsed time=  0.015424489974975586 s
<------------------------------->


Actual:  b685n
Predicted: n6357 

Percentage Similarity=  40.0
Elapsed time=  0.014712810516357422 s
<------------------------------->


Actual:  dn26n
Predicted: dn257 

Percentage Similarity=  60.0
Elapsed time=  0.014897346496582031 s
<------------------------------->


Actual:  6fgdw
Predicted: 5fyxy 

Percentage Similarity=  20.0
Elapsed time=  0.015624523162841797 s
<------------------------------->


Actual:  c4bgd
Predicted: e4bnd 

Percentage Similarity=  60.0
Elapsed time=  0.01510930061340332 s
<------------------------------->


Actual:  byc82
Predicted: byc32 

Percentage Similarity= 

Predicted: 37c22 

Percentage Similarity=  60.0
Elapsed time=  0.02223944664001465 s
<------------------------------->


Actual:  8gmc4
Predicted: 8gnc4 

Percentage Similarity=  80.0
Elapsed time=  0.020131826400756836 s
<------------------------------->


Actual:  4dgf7
Predicted: 4d5f7 

Percentage Similarity=  80.0
Elapsed time=  0.018621206283569336 s
<------------------------------->


Actual:  44ype
Predicted: 53yfe 

Percentage Similarity=  40.0
Elapsed time=  0.020862817764282227 s
<------------------------------->


Actual:  2yggg
Predicted: 2yggc 

Percentage Similarity=  80.0
Elapsed time=  0.020200014114379883 s
<------------------------------->


Actual:  8d8ep
Predicted: 8deep 

Percentage Similarity=  80.0
Elapsed time=  0.019811391830444336 s
<------------------------------->


Actual:  de45x
Predicted: 5ew2x 

Percentage Similarity=  40.0
Elapsed time=  0.01803112030029297 s
<------------------------------->


Actual:  82fx2
Predicted: 8exx2 

Percentage Similarity=  

Predicted: f4ny3 

Percentage Similarity=  40.0
Elapsed time=  0.025189638137817383 s
<------------------------------->


Actual:  ennmm
Predicted: epypc 

Percentage Similarity=  20.0
Elapsed time=  0.019803762435913086 s
<------------------------------->


Actual:  5yxgp
Predicted: 82ycp 

Percentage Similarity=  20.0
Elapsed time=  0.01844477653503418 s
<------------------------------->


Actual:  e72cd
Predicted: e727d 

Percentage Similarity=  80.0
Elapsed time=  0.018366098403930664 s
<------------------------------->


Actual:  cpe63
Predicted: 2egc5 

Percentage Similarity=  0.0
Elapsed time=  0.017414569854736328 s
<------------------------------->


Actual:  467d5
Predicted: e62x5 

Percentage Similarity=  40.0
Elapsed time=  0.01788020133972168 s
<------------------------------->


Actual:  5wddw
Predicted: 5xcex 

Percentage Similarity=  20.0
Elapsed time=  0.015427589416503906 s
<------------------------------->


Actual:  d236n
Predicted: d2327 

Percentage Similarity=  6

Predicted: 5mnyn 

Percentage Similarity=  60.0
Elapsed time=  0.01674485206604004 s
<------------------------------->


Actual:  6fn84
Predicted: 5fnxg 

Percentage Similarity=  40.0
Elapsed time=  0.016611337661743164 s
<------------------------------->


Actual:  5mcy7
Predicted: 5x27f 

Percentage Similarity=  20.0
Elapsed time=  0.01795482635498047 s
<------------------------------->


Actual:  cndmc
Predicted: cncpc 

Percentage Similarity=  60.0
Elapsed time=  0.015161514282226562 s
<------------------------------->


Actual:  37d52
Predicted: 32g85 

Percentage Similarity=  20.0
Elapsed time=  0.014934301376342773 s
<------------------------------->


Actual:  5325m
Predicted: 57452 

Percentage Similarity=  40.0
Elapsed time=  0.01559591293334961 s
<------------------------------->


Actual:  b5fm7
Predicted: 22xyy 

Percentage Similarity=  0.0
Elapsed time=  0.01574254035949707 s
<------------------------------->


Actual:  3fbxd
Predicted: 3fcyg 

Percentage Similarity=  40.

Predicted: 2m537 

Percentage Similarity=  60.0
Elapsed time=  0.017885923385620117 s
<------------------------------->


Actual:  5nggg
Predicted: 5pxpc 

Percentage Similarity=  20.0
Elapsed time=  0.017713069915771484 s
<------------------------------->


Actual:  8d4wm
Predicted: 8x5xm 

Percentage Similarity=  40.0
Elapsed time=  0.016050338745117188 s
<------------------------------->


Actual:  4dw3w
Predicted: xcx2y 

Percentage Similarity=  0.0
Elapsed time=  0.014712333679199219 s
<------------------------------->


Actual:  77387
Predicted: 77372 

Percentage Similarity=  60.0
Elapsed time=  0.015913963317871094 s
<------------------------------->


Actual:  728n8
Predicted: 752y3 

Percentage Similarity=  20.0
Elapsed time=  0.015113115310668945 s
<------------------------------->


Actual:  cffp4
Predicted: effx4 

Percentage Similarity=  60.0
Elapsed time=  0.015584230422973633 s
<------------------------------->


Actual:  e6b7y
Predicted: e5g2y 

Percentage Similarity= 

<------------------------------->


Actual:  2356g
Predicted: 755xc 

Percentage Similarity=  20.0
Elapsed time=  0.01728677749633789 s
<------------------------------->


Actual:  e3cfe
Predicted: e3cfe 

Percentage Similarity=  100.0
Elapsed time=  0.017659902572631836 s
<------------------------------->


Actual:  32cnn
Predicted: 322yc 

Percentage Similarity=  40.0
Elapsed time=  0.01563572883605957 s
<------------------------------->


Actual:  28348
Predicted: 25xbx 

Percentage Similarity=  20.0
Elapsed time=  0.01501774787902832 s
<------------------------------->


Actual:  bgxcd
Predicted: bg2cc 

Percentage Similarity=  60.0
Elapsed time=  0.015516281127929688 s
<------------------------------->


Actual:  2en7g
Predicted: 5cc2n 

Percentage Similarity=  0.0
Elapsed time=  0.015014886856079102 s
<------------------------------->


Actual:  b4ndb
Predicted: b4ndb 

Percentage Similarity=  100.0
Elapsed time=  0.014805316925048828 s
<------------------------------->


Actual: